# 语料

In [21]:
import pandas as pd

df_txt = pd.read_csv('../data/kapok.txt') 

In [22]:
import re
import jieba
import jieba.analyse

def segment(text):
    jieba.analyse.set_stop_words('/home/its/CodeReview/LLM/data/baidu_stopwords.txt')
    words = jieba.cut(text)
    words = [word for word in jieba.cut(text) ]
    return words

In [50]:
import re

list_word = df_txt.iloc[:,0].apply(segment).to_list()
#.apply(lambda txt: ''.join(re.findall(r'[\u4e00-\u9fa5]+', txt)))

[['【', '作者', '】', '舒婷', ' ', '【', '朝代', '】', '现代'],
 ['我', '如果', '爱', '你', '—', '—'],
 ['绝不', '像', '攀援', '的', '凌霄花', '，'],
 ['借', '你', '的', '高枝', '炫耀', '自己', '；'],
 ['我', '如果', '爱', '你', '—', '—'],
 ['绝不', '学', '痴情', '的', '鸟儿', '，'],
 ['为', '绿荫', '重复', '单调', '的', '歌曲', '；'],
 ['也', '不止', '像', '泉源', '，'],
 ['常年', '送来', '清凉', '的', '慰藉', '；'],
 ['也', '不止', '像', '险峰', '，'],
 ['增加', '你', '的', '高度', '，', '衬托', '你', '的', '威仪', '。'],
 ['甚至', '日光', '，'],
 ['甚至', '春雨', '。'],
 ['不', '，', '这些', '都', '还', '不够', '！'],
 ['我', '必须', '是', '你', '近旁', '的', '一株', '木棉', '，'],
 ['作为', '树', '的', '形象', '和', '你', '站', '在', '一起', '。'],
 ['根', '，', '紧握', '在', '地下', '；'],
 ['叶', '，', '相触', '在', '云里', '。'],
 ['每', '一阵风', '过', '，'],
 ['我们', '都', '互相', '致意', '，'],
 ['但', '没有', '人', '，'],
 ['听懂', '我们', '的', '言语', '。'],
 ['你', '有', '你', '的', '铜枝', '铁干', '，'],
 ['像', '刀', '，', '像', '剑', '，', '也', '像', '戟', '；'],
 ['我', '有', '我', '红硕', '的', '花朵', '，'],
 ['像', '沉重', '的', '叹息', '，'],
 ['又', '像', '英勇', '的', '火炬', '。'],
 ['

# 词袋

In [89]:
# 生成一本字典
from gensim import corpora

dictionary = corpora.Dictionary(list_word)
# 把字典存储下来，可以在以后直接导入
dictionary.save(fname_or_handle='/home/its/CodeReview/LLM/data/love.dict')

In [70]:
#dictionary.filter_extremes(no_below=20, no_above=0.5)  # 删掉只在不超过20个文本中出现过的词，删掉在50%及以上的文本都出现了的词
# dictionary.filter_tokens(['一个'])  # 这个函数可以直接删除指定的词
dictionary.compactify()  # 去掉因删除词汇而出现的空白
#dictionary.filter_n_most_frequent(200)

## 语料库向量化

In [71]:
corpus = [dictionary.doc2bow(text) for text in list_word]
 # 存储语料库
corpora.MmCorpus.serialize('../data/corpus_bow.mm', corpus) 

In [81]:
corpus[8]

[(15, 1), (22, 1), (34, 1), (35, 1), (36, 1), (37, 1)]

In [82]:
from gensim.models import LdaModel


lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    chunksize=2000,
    alpha='auto',
    eta='auto',
    iterations=500,
    num_topics=3,
    passes=20,
    eval_every=None
)

model.save(fname='../mold/love.gs')

In [77]:
lda_model.print_topics(10)

[(0,
  '0.104*"，" + 0.053*"像" + 0.053*"的" + 0.031*"；" + 0.031*"也" + 0.017*"我们" + 0.017*"、" + 0.017*"是" + 0.017*"绝不" + 0.017*"不止"'),
 (1,
  '0.043*"的" + 0.030*"【" + 0.030*"】" + 0.030*"你" + 0.030*"，" + 0.017*"爱" + 0.017*"现代" + 0.017*"作者" + 0.017*"不仅" + 0.017*"高枝"'),
 (2,
  '0.062*"的" + 0.054*"。" + 0.054*"你" + 0.047*"，" + 0.047*"—" + 0.032*"我" + 0.025*"在" + 0.025*"爱" + 0.017*"、" + 0.017*"我们"')]

In [111]:
lda_model.top_topics(corpus, topn=8)

[([(0.08307864, '的'),
   (0.07270475, '，'),
   (0.05706723, '你'),
   (0.03003431, '我'),
   (0.030030088, '。'),
   (0.02102469, '有'),
   (0.021024216, '；'),
   (0.012014075, '增加')],
  -8.644459130211269),
 ([(0.051997684, '的'),
   (0.040000077, '；'),
   (0.039999485, '你'),
   (0.027999258, '、'),
   (0.02798469, '，'),
   (0.016000262, '作为'),
   (0.016000252, '和'),
   (0.016000241, '站')],
  -11.930959820039257),
 ([(0.08041004, '，'),
   (0.052857444, '像'),
   (0.040419374, '—'),
   (0.03476327, '的'),
   (0.034043096, '。'),
   (0.021276888, '爱'),
   (0.021275511, '也'),
   (0.021274585, '我们')],
  -11.976786792459153)]

In [109]:
import pyLDAvis.gensim_models as gensimvis
from pyLDAvis import gensim

pyLDAvis.enable_notebook

vis = gensimvis.prepare(lda_model, corpus, dictionary, sort_topics=False)

pyLDAvis.display(vis)
pyLDAvis.save_html(data=vis, fileobj='../data/lda_gensim.html')